# Explore Codebreaker 2018 Blockchain

This notebook can be used to poke around CodeBreaker-2018 blockchain.


## Initial setup

Let's import some libraries

In [1]:
import json
import os

from web3 import Web3
from solc import compile_source
from web3.contract import ConciseContract
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import pandas as pd
import urllib
from datetime import datetime

## Settings

In [2]:
url = 'https://codebreaker.ltsnet.net/eth/8e415a1477b0a587/d6389b319e1a35111021d019d0a494158dab3c2308a0a8a21cec8124270c766b'


Is everything OK?

In [3]:
web3 = Web3(Web3.HTTPProvider(url, request_kwargs={'verify':False}))
web3.isConnected()

True

Let's define some useful functions

In [4]:
def load_contract(w3, name):   # Load compiled contract
    location = compiled_contracts[name]
    with urllib.request.urlopen(location) as url:
        info = json.loads(url.read().decode())
        return w3.eth.contract(abi=info['abi'], bytecode=info['bytecode'])
    
def htoeth(bal):   # Convert hext to ether amount
    return int(bal.hex(),16)/(10**18)

def addr(hb_val):  # Convert hex value to checksumed address
    return chksum(hb_val.hex()[-40:])

Some shortcuts

In [5]:
w3 = web3
eth = web3.eth
gb = eth.getBalance
gs = eth.getStorageAt
chksum = w3.toChecksumAddress

We also going to need compiled versions of the ethereum smart contracts given to us

In [6]:
compiled_contracts = {
'ransom' : 'http://bit.ly/2Cgac0z',
'escrow' : 'http://bit.ly/2shvare',
'registry' : 'http://bit.ly/2RBBAjl'
}        

# Chain Info
Now we should be able to get basic info from the chain

In [7]:
w3.eth.blockNumber

1922232

In [8]:
w3.eth.gasPrice/(10**9)

1.0

It's nice that Python supports big number natively, so we don't need to resort to the uglines of BigNumber.js

# Constants

Let's enter some constants given to us and create placeholder variables for some more

In [9]:
# Accounts
owner=None
oracle=None
victim="0xc09812b25cbeBC4D688696e541Ce5E8C410380BF"

# Contracts
registry=None
escrow="0x2268A64F65318bCDdFef1A799c61c1E3073FeDa5"
ransom="0xc09812b25cbeBC4D688696e541Ce5E8C410380BF"

myCID = "0xb784c8325a15d7b7d62d4ded79b86b08fd0cbc8ed0099fee200b55ef8791eae6"

# Escrow contract examination

Let's examine the internal state of Escrow contract

In [10]:
owner = addr(gs(escrow, 0))  # Owner address
owner

'0x63D85378Eb4D57C4aE14f6a39B05e495dE08B1a8'

In [11]:
ownerBal = gb(owner)/(10**18) # Owner Ethereum wallet balance
ownerBal

9.999999999999982e+17

In [12]:
escrowOwnerBal = gs(escrow, 1)  # How much ether owner can withdraw
escrowOwnerBal = htoeth(escrowOwnerBal)
escrowOwnerBal

300.0

In [13]:
escrowContractBal = gb(escrow)/(10**18) # How much does ether does Escrow contract holds
escrowContractBal

300.0

In [14]:
registry = addr(gs(escrow, 2)) # What's Registry address
registry

'0x6b4e1E0755a93C6B60851dC72f4E32fa18ac6828'

In [15]:
oracle = addr(gs(escrow,10))  # What's oracle address
oracle

'0x191B13d28Df6b574275405e485dfC0F6794AD831'

### All known ransom contracts

Now lets' enumerate all Ransom contracts that Escrow knows about

In [16]:
index = "%064x" % 9
index = web3.sha3(hexstr=index)
ransoms = [addr(gs(escrow,int(index.hex(),16)+i)) for i in range(0,10)]
ransoms

['0xBC8794A37A428b86542A587c047A01cd18B9D826',
 '0xc6dE5B74d100168Bf368F63fd1cfd0B46B2A8772',
 '0x049515Ef597Ce9F2237eCB3Fa9c15bFad2BCAd90',
 '0xA6D7FF4A4e4480c009c32F28981a8ac64943c006',
 '0x592cbD96f1209Da30518110ec036F864Eeed0C4D',
 '0xd95028DdB9bef50b78256D9053CBdB92cCe9fEeA',
 '0x71480981B2FaE54c5430Ba69bCBB41496a879C84',
 '0xdC782dBd6aAB2BE81f884F81130Cf70Cc3bDEcb6',
 '0xc188D7c8FE22D73c014cE57DBa6096fEa419ABC2',
 '0xc09812b25cbeBC4D688696e541Ce5E8C410380BF']

### Victim IDs

We got all Ransom contract addresses we can now get the victim IDs

In [17]:
vic_ids = []
for rr in ransoms:
    index = "%064x" % 6
    r = rr[2:].lower()
    index = ('0' * 24) + r + index
    index = web3.sha3(hexstr=index)
    vic_id = gs(escrow,int(index.hex(),16))
    vic_ids.append(vic_id.hex())
vic_ids

['0x739f2fbe73b39319a3b582b6f1ea6f85a5b97f9e385961c06e7e0c8b6c124f1f',
 '0x8d4e739e63b58576664c5b2ecd068e59322bb6109629b5798795bafdb1e14c9f',
 '0x593a32f34377c6e4bb1b737a349bb9d73ab1200ff74885dee59170cb88319056',
 '0xd37dc467c8f40f2fd6892604e7cfd0aa1020d7bf86e153f435c99636b11c24a8',
 '0xa059db461066052570e661a54d0bd5613b0244bfc94fa2c88a92bab76576e557',
 '0xdcde9c6864cc9ff032253750917ae807fd67eda94b1e9a8029fa4abd10f9cda6',
 '0x8de57837b523d2dd8c0f4e5ee1fda7e976ea9eeb171ff80297aa6ba9ff41158f',
 '0x671538f3cbb58f0d532f24498241e019eeabc1fead783cf9565b635cf1241008',
 '0xd0f2495ba15d13f347b5bb4f45650d843f64d213b1d4122698c2c29c6086c6da',
 '0x58cc997ff6b6cff27c6dd2270709913a63d0be99ce40b0bbd78266cd02c95d8f']

### Who paid

Now let's see who paid

In [18]:
vic_paid_addr = {}
for vv in vic_ids:
    index = "%064x" % 3
    index = vv[2:] + index
    index = web3.sha3(hexstr=index)
    paid = gs(escrow,int(index.hex(),16))
    if(int(paid.hex(),16) != 0):
        vic_paid_addr[vv] = addr(paid)
vic_paid_addr

{'0x739f2fbe73b39319a3b582b6f1ea6f85a5b97f9e385961c06e7e0c8b6c124f1f': '0xa411DF1aE52E324cb3A123bf4888233024926CFd',
 '0x8de57837b523d2dd8c0f4e5ee1fda7e976ea9eeb171ff80297aa6ba9ff41158f': '0xfACc57958A577003EADA8b8Ff9794FC1535D38dC',
 '0xd37dc467c8f40f2fd6892604e7cfd0aa1020d7bf86e153f435c99636b11c24a8': '0xfcECA7d3bF63D243fc885B8f53526Ff5ed891eb4'}

### Let's put it all together

And now let's dump 'victim information table' from the Escrow contract

In [19]:
vic_map = {}
for vv in vic_ids:
    index = "%064x" % 4
    index = vv[2:] + index
    index = web3.sha3(hexstr=index)
    mmb = [gs(escrow,int(index.hex(),16)+i) for i in range(0,4)]
    mmb = (mmb[0].hex(),int(mmb[1].hex(),16)/(10**18),addr(mmb[2]),addr(mmb[3]))
    vic_map[mmb[0]] = {'ransom_amt':mmb[1],'victim_addr':mmb[2],'ransom_addr':mmb[3]}
vic_map
vic_map_df = pd.DataFrame.from_dict(vic_map,orient='index')
vic_map_df

,victim_addr,ransom_amt,ransom_addr
0x58cc997ff6b6cff27c6dd2270709913a63d0be99ce40b0bbd78266cd02c95d8f,0x27cBF212B3de98719bC5FB682bB0f8383e8177E4,100.0,0xc09812b25cbeBC4D688696e541Ce5E8C410380BF
0x593a32f34377c6e4bb1b737a349bb9d73ab1200ff74885dee59170cb88319056,0xd8742509518b95183CFf255E38A5a0685C1FBf52,100.0,0x049515Ef597Ce9F2237eCB3Fa9c15bFad2BCAd90
0x671538f3cbb58f0d532f24498241e019eeabc1fead783cf9565b635cf1241008,0x50AF09c4820c8f29E7c13E524d4AE62c11C364b7,100.0,0xdC782dBd6aAB2BE81f884F81130Cf70Cc3bDEcb6
0x739f2fbe73b39319a3b582b6f1ea6f85a5b97f9e385961c06e7e0c8b6c124f1f,0xa411DF1aE52E324cb3A123bf4888233024926CFd,100.0,0xBC8794A37A428b86542A587c047A01cd18B9D826
0x8d4e739e63b58576664c5b2ecd068e59322bb6109629b5798795bafdb1e14c9f,0x5156262D2a917bB2856D6164d67BcF12dc1ABd5B,100.0,0xc6dE5B74d100168Bf368F63fd1cfd0B46B2A8772
0x8de57837b523d2dd8c0f4e5ee1fda7e976ea9eeb171ff80297aa6ba9ff41158f,0xfACc57958A577003EADA8b8Ff9794FC1535D38dC,100.0,0x71480981B2FaE54c5430Ba69bCBB41496a879C84
0xa059db461066052570e661a54d0bd5613b0244bfc94fa2c88a92bab76576e557,0xA72e9A9Bd21aA74C8857625CfC0C9082d477d6Fa,100.0,0x592cbD96f1209Da30518110ec036F864Eeed0C4D
0xd0f2495ba15d13f347b5bb4f45650d843f64d213b1d4122698c2c29c6086c6da,0x146676783d93d8b7CFd3AF243B74dd406f38EcD4,100.0,0xc188D7c8FE22D73c014cE57DBa6096fEa419ABC2
0xd37dc467c8f40f2fd6892604e7cfd0aa1020d7bf86e153f435c99636b11c24a8,0xfcECA7d3bF63D243fc885B8f53526Ff5ed891eb4,100.0,0xA6D7FF4A4e4480c009c32F28981a8ac64943c006
0xdcde9c6864cc9ff032253750917ae807fd67eda94b1e9a8029fa4abd10f9cda6,0xd8b09121DF17d2C622E47c1C83D8022bc4a66A5D,100.0,0xd95028DdB9bef50b78256D9053CBdB92cCe9fEeA


### Reveal decryption keys

Now let's take a look at the decryption keys sent by the oracle in response to the valid ransom payment

In [20]:
dec_keys = {}
for vv in vic_paid_addr.keys():
    index = "%064x" % 5
    index = vv[2:] + index
    index = web3.sha3(hexstr=index)
    dec_key = gs(escrow,int(index.hex(),16))
    dec_keys[vv] = dec_key.hex()
dec_keys

{'0x739f2fbe73b39319a3b582b6f1ea6f85a5b97f9e385961c06e7e0c8b6c124f1f': '0x7bf7f148c5182b5a91a138eb0270834f5f3dd25d5823b900259b427def54eb2a',
 '0x8de57837b523d2dd8c0f4e5ee1fda7e976ea9eeb171ff80297aa6ba9ff41158f': '0x8aa618d5972eef00f765d29382ab8d00ff9720d2d97c64f3b7a1ffa4fe144f2f',
 '0xd37dc467c8f40f2fd6892604e7cfd0aa1020d7bf86e153f435c99636b11c24a8': '0x9d2268159171a816bc9d6c2487880f9c6e1f4192951e9308a2a362c98392549d'}

### The rest of the Escrow datastructures

From the source we see that the escrow contract also has two data structures called `escrowMap` and `encFileMap`. The comments say that these are for keeping track of amount of ransom received from the victim and the encrypted file sent with the ransom.   

Let's look at their contents

In [21]:
# Amount received
esc_balance = {}
for vv in vic_ids:
    index = "%064x" % 7
    index = vv[2:] + index
    index = web3.sha3(hexstr=index)
    dec_key = gs(escrow,int(index.hex(),16))
    esc_balance[vv] = int(dec_key.hex(),16)
esc_balance

{'0x58cc997ff6b6cff27c6dd2270709913a63d0be99ce40b0bbd78266cd02c95d8f': 0,
 '0x593a32f34377c6e4bb1b737a349bb9d73ab1200ff74885dee59170cb88319056': 0,
 '0x671538f3cbb58f0d532f24498241e019eeabc1fead783cf9565b635cf1241008': 0,
 '0x739f2fbe73b39319a3b582b6f1ea6f85a5b97f9e385961c06e7e0c8b6c124f1f': 0,
 '0x8d4e739e63b58576664c5b2ecd068e59322bb6109629b5798795bafdb1e14c9f': 0,
 '0x8de57837b523d2dd8c0f4e5ee1fda7e976ea9eeb171ff80297aa6ba9ff41158f': 0,
 '0xa059db461066052570e661a54d0bd5613b0244bfc94fa2c88a92bab76576e557': 0,
 '0xd0f2495ba15d13f347b5bb4f45650d843f64d213b1d4122698c2c29c6086c6da': 0,
 '0xd37dc467c8f40f2fd6892604e7cfd0aa1020d7bf86e153f435c99636b11c24a8': 0,
 '0xdcde9c6864cc9ff032253750917ae807fd67eda94b1e9a8029fa4abd10f9cda6': 0}

In [22]:
# Encrypted file map
enc_file_map = {}
for vv in vic_ids:
    index = "%064x" % 8
    index = vv[2:] + index
    index = web3.sha3(hexstr=index)
    enc_file = gs(escrow,int(index.hex(),16))
    enc_file_map[vv] = enc_file.hex()
enc_file_map

{'0x58cc997ff6b6cff27c6dd2270709913a63d0be99ce40b0bbd78266cd02c95d8f': '0x0000000000000000000000000000000000000000000000000000000000000000',
 '0x593a32f34377c6e4bb1b737a349bb9d73ab1200ff74885dee59170cb88319056': '0x0000000000000000000000000000000000000000000000000000000000000000',
 '0x671538f3cbb58f0d532f24498241e019eeabc1fead783cf9565b635cf1241008': '0x0000000000000000000000000000000000000000000000000000000000000000',
 '0x739f2fbe73b39319a3b582b6f1ea6f85a5b97f9e385961c06e7e0c8b6c124f1f': '0x0000000000000000000000000000000000000000000000000000000000000000',
 '0x8d4e739e63b58576664c5b2ecd068e59322bb6109629b5798795bafdb1e14c9f': '0x0000000000000000000000000000000000000000000000000000000000000000',
 '0x8de57837b523d2dd8c0f4e5ee1fda7e976ea9eeb171ff80297aa6ba9ff41158f': '0x0000000000000000000000000000000000000000000000000000000000000000',
 '0xa059db461066052570e661a54d0bd5613b0244bfc94fa2c88a92bab76576e557': '0x0000000000000000000000000000000000000000000000000000000000000000',
 '0xd0f2495ba

These seem to be empty... let's take a note of that and move on

# Examine Registry

Now let's look at the data stored in the registry contract 

In [23]:
r_owner = addr(gs(registry, 1))   ## Owner address
r_owner

'0xE4F2555B638Ce2E8df8A3E5DB32EDE37D4698231'

In [24]:
victim_count = int(gs(registry, 2).hex(),16) ## How many victims
victim_count

50502

In [25]:
r_oracle = addr(gs(registry, 3))  ## Oracle's address
r_oracle

'0x191B13d28Df6b574275405e485dfC0F6794AD831'

In [26]:
pending_auth = int(gs(registry, 5).hex(),16)  ## Number of pending auth
pending_auth

268435715

The above number is pretty high.  Turns out there is a bug in the registry that makes this number increase forever

In [27]:
max_pending_auth = int(gs(registry, 6).hex(),16)  # Max Pending Authentication
max_pending_auth

0

Due to the bug above,  looks like they have disabled the throttling mechanism

### Larger data structures

Now let's take a look at the larger data structures.  `vic_map` and `vic_auth_map`

In [28]:
rg_vic_map = {}
for vv in vic_ids:
    index = "%064x" % 0
    index = vv[2:] + index
    index = web3.sha3(hexstr=index)
    mmb = [gs(registry,int(index.hex(),16)+i) for i in range(0,3)]
    mmb = (mmb[0].hex(),addr(mmb[1]),addr(mmb[2]))
    rg_vic_map[mmb[0]] = {'escrow_addr':mmb[2],'ransom_addr':mmb[1]}
rg_vic_map_df = pd.DataFrame.from_dict(rg_vic_map,orient='index')
rg_vic_map_df

,escrow_addr,ransom_addr
0x58cc997ff6b6cff27c6dd2270709913a63d0be99ce40b0bbd78266cd02c95d8f,0x2268A64F65318bCDdFef1A799c61c1E3073FeDa5,0xc09812b25cbeBC4D688696e541Ce5E8C410380BF
0x593a32f34377c6e4bb1b737a349bb9d73ab1200ff74885dee59170cb88319056,0x2268A64F65318bCDdFef1A799c61c1E3073FeDa5,0x049515Ef597Ce9F2237eCB3Fa9c15bFad2BCAd90
0x671538f3cbb58f0d532f24498241e019eeabc1fead783cf9565b635cf1241008,0x2268A64F65318bCDdFef1A799c61c1E3073FeDa5,0xdC782dBd6aAB2BE81f884F81130Cf70Cc3bDEcb6
0x739f2fbe73b39319a3b582b6f1ea6f85a5b97f9e385961c06e7e0c8b6c124f1f,0x2268A64F65318bCDdFef1A799c61c1E3073FeDa5,0xBC8794A37A428b86542A587c047A01cd18B9D826
0x8d4e739e63b58576664c5b2ecd068e59322bb6109629b5798795bafdb1e14c9f,0x2268A64F65318bCDdFef1A799c61c1E3073FeDa5,0xc6dE5B74d100168Bf368F63fd1cfd0B46B2A8772
0x8de57837b523d2dd8c0f4e5ee1fda7e976ea9eeb171ff80297aa6ba9ff41158f,0x2268A64F65318bCDdFef1A799c61c1E3073FeDa5,0x71480981B2FaE54c5430Ba69bCBB41496a879C84
0xa059db461066052570e661a54d0bd5613b0244bfc94fa2c88a92bab76576e557,0x2268A64F65318bCDdFef1A799c61c1E3073FeDa5,0x592cbD96f1209Da30518110ec036F864Eeed0C4D
0xd0f2495ba15d13f347b5bb4f45650d843f64d213b1d4122698c2c29c6086c6da,0x2268A64F65318bCDdFef1A799c61c1E3073FeDa5,0xc188D7c8FE22D73c014cE57DBa6096fEa419ABC2
0xd37dc467c8f40f2fd6892604e7cfd0aa1020d7bf86e153f435c99636b11c24a8,0x2268A64F65318bCDdFef1A799c61c1E3073FeDa5,0xA6D7FF4A4e4480c009c32F28981a8ac64943c006
0xdcde9c6864cc9ff032253750917ae807fd67eda94b1e9a8029fa4abd10f9cda6,0x2268A64F65318bCDdFef1A799c61c1E3073FeDa5,0xd95028DdB9bef50b78256D9053CBdB92cCe9fEeA


Looks like `vic_map` maintains linkage between *Escrow* and *Ransom* contracts

In [29]:
rg_vic_auth_map = {}
my_vicids = [
    '0xdcde9c6864cc9ff032253750917ae807fd67eda94b1e9a8029fa4abd10f9cda6',
    '0x8de57837b523d2dd8c0f4e5ee1fda7e976ea9eeb171ff80297aa6ba9ff41158f',
    '0x671538f3cbb58f0d532f24498241e019eeabc1fead783cf9565b635cf1241008'
]
for vv in my_vicids:
    index = "%064x" % 4
    index = vv[2:] + index
    index = web3.sha3(hexstr=index)
    mmb = [gs(registry,int(index.hex(),16)+i) for i in range(0,3)]
    mmb = (mmb[0].hex(),addr(mmb[1]),addr(mmb[2]))
    rg_vic_auth_map[mmb[0]] = {'escrow_addr':mmb[2],'ransom_addr':mmb[1]}
rg_vic_auth_map_df = pd.DataFrame.from_dict(rg_vic_auth_map,orient='index')
rg_vic_auth_map_df

,escrow_addr,ransom_addr
0x0000000000000000000000000000000000000000000000000000000000000000,0x0000000000000000000000000000000000000000,0x0000000000000000000000000000000000000000


`vic_auth_map` temporarily holds *Ransom* to *Escrow* linkage until it has been confirmed by off the chain oracle

# Examine accounts

Let's look at the balances of the accounts associated with the victims

In [30]:
{a:gb(a)/(10**18) for a in list(vic_map_df.victim_addr.values)}

{'0x146676783d93d8b7CFd3AF243B74dd406f38EcD4': 125.0,
 '0x27cBF212B3de98719bC5FB682bB0f8383e8177E4': 25.0,
 '0x50AF09c4820c8f29E7c13E524d4AE62c11C364b7': 125.0,
 '0x5156262D2a917bB2856D6164d67BcF12dc1ABd5B': 125.0,
 '0xA72e9A9Bd21aA74C8857625CfC0C9082d477d6Fa': 125.0,
 '0xa411DF1aE52E324cb3A123bf4888233024926CFd': 24.989310376,
 '0xd8742509518b95183CFf255E38A5a0685C1FBf52': 125.0,
 '0xd8b09121DF17d2C622E47c1C83D8022bc4a66A5D': 125.0,
 '0xfACc57958A577003EADA8b8Ff9794FC1535D38dC': 24.989310376,
 '0xfcECA7d3bF63D243fc885B8f53526Ff5ed891eb4': 24.989310376}

We see that most victims who havent' paid the ransom yet have a balance of 125 ETH.  The vicitm's who did pay, have their account balances reduced by 100 ether plus some small amount of gas money.  Our account has only 25 ETH so we can't pay the ransom, we have to exploit the blockchain to get our decryption key out of it

Let's check how many transactions were performed by each victim

In [31]:
{a:web3.eth.getTransactionCount(a) for a in list(vic_map_df.victim_addr.values)}

{'0x146676783d93d8b7CFd3AF243B74dd406f38EcD4': 0,
 '0x27cBF212B3de98719bC5FB682bB0f8383e8177E4': 0,
 '0x50AF09c4820c8f29E7c13E524d4AE62c11C364b7': 0,
 '0x5156262D2a917bB2856D6164d67BcF12dc1ABd5B': 0,
 '0xA72e9A9Bd21aA74C8857625CfC0C9082d477d6Fa': 0,
 '0xa411DF1aE52E324cb3A123bf4888233024926CFd': 1,
 '0xd8742509518b95183CFf255E38A5a0685C1FBf52': 0,
 '0xd8b09121DF17d2C622E47c1C83D8022bc4a66A5D': 0,
 '0xfACc57958A577003EADA8b8Ff9794FC1535D38dC': 1,
 '0xfcECA7d3bF63D243fc885B8f53526Ff5ed891eb4': 1}

Again we see that the accounts who paid executed one transaction on the blockchain and the rest didn't do anything 

Let's check transaction count by the Attackers accounts

In [32]:
(web3.eth.getTransactionCount(escrow),
 web3.eth.getTransactionCount(registry),
 web3.eth.getTransactionCount(oracle), 
 web3.eth.getTransactionCount(owner), 
 web3.eth.getTransactionCount(r_owner))

(1, 1, 253205, 55000, 7)

We see that escrow and registry contracts have only one transaction posted by them.  That's when these contracts were first created.  And oracle and owner have a lot of transactions posted.  These snipped will be useful for us later on

# Blockchain events

In addition to contract storage an balances there is another important part of the blockchain data, the logged events.
Logged events are pieces of data written to the blockchain when contract executes 'emit' statement.  Unlike contract storage variables,  the values the events do not overwrite their data. So we can query the blockchain for all the events emitted by the contract

First we need to load the compiled code for our contracts

In [33]:
escrowCode = load_contract(web3, 'escrow')
ransomCode = load_contract(web3, 'ransom')
registryCode = load_contract(web3, 'registry')

Once we have the compiled code we can query the blockchain for the events emitted by contracts

### Auth events from registry

When new Ransom contract is created it requests registry to attach it to the Escrow.  The Registry contract in turn posts an *AuthEvent* to the blockchain, and waits for the response to it from the oracle.   Let's take a look at some of  these  events

In [34]:
flt = registryCode.events.AuthEvent.createFilter(fromBlock=1747955,toBlock=web3.eth.blockNumber)
ent = flt.get_all_entries()
ent = ent[:10]

In [35]:
auth_events =  {n: {
    'id':"%x" % e.args['id'],
    'block': e.blockNumber,
    'tidx': e.transactionIndex,
    'origin': e.args['origin'],
    'ts': datetime.utcfromtimestamp(web3.eth.getBlock(e.blockNumber)['timestamp']),
    'ransom': e.args['ransomAddr'],
    'OTP': e.args['authToken'] } for n,e in enumerate(ent) }
auth_events_df = pd.DataFrame.from_dict(auth_events,orient='index')
auth_events_df

,ransom,OTP,ts,block,tidx,id,origin
0,0xA20A4Ed8c997668f8D9947F4C414D035A81B6be1,668748,2018-12-31 05:50:50,1748035,0,f63133962cabe6c46373d552dc8ae2930987f559fb70ea...,0xC48F3e7De3bf905660EDd42C41E460555CefE2fE
1,0x1E6a301A8f9Fd1400a65b3465CEfFb12746E40b6,551095,2018-12-31 06:01:49,1748160,0,f63133962cabe6c46373d552dc8ae2930987f559fb70ea...,0xC48F3e7De3bf905660EDd42C41E460555CefE2fE
2,0x9f6a7320bD0b3717d25Ea85f4B96c2ebaA12CC7a,63527,2018-12-31 06:05:50,1748205,0,b784c8325a15d7b7d62d4ded79b86b08fd0cbc8ed0099f...,0xC48F3e7De3bf905660EDd42C41E460555CefE2fE
3,0x78b24e154827822313EEc48E06eefAC47296bca3,381712,2018-12-31 21:14:46,1758443,0,b784c8325a15d7b7d62d4ded79b86b08fd0cbc8ed0099f...,0xC48F3e7De3bf905660EDd42C41E460555CefE2fE
4,0xBa94bC8dd2201344a0c3A4276427AB2F80d434c7,762801,2019-01-02 01:47:14,1777721,0,691044a169e7fdda738ba4c7cc6d31b88d298546a326b7...,0x734eF4152bC53fC4559709BF11267F251e7531Aa
5,0x7DEE6f4CB1167FaAF5c22d2cCe09d8C5F521F0EC,356934,2019-01-02 06:55:12,1781190,0,691044a169e7fdda738ba4c7cc6d31b88d298546a326b7...,0x734eF4152bC53fC4559709BF11267F251e7531Aa
6,0xF08803AeEdE010e5A3cCE741819b8073208f9002,637175,2019-01-02 21:29:22,1791047,0,691044a169e7fdda738ba4c7cc6d31b88d298546a326b7...,0x734eF4152bC53fC4559709BF11267F251e7531Aa
7,0x8e618127e04550598Db4508F5f3A138CF6446Ca8,981744,2019-01-02 23:58:47,1792733,0,691044a169e7fdda738ba4c7cc6d31b88d298546a326b7...,0x734eF4152bC53fC4559709BF11267F251e7531Aa
8,0x0c910A037fC1fD8e057601f402595389FeEF5b8D,123703,2019-01-03 21:50:01,1807502,0,4c326dd7b5efb8b3615574b92502f997954f2f54e7c7f2...,0x13495A46D4de87020753e3178462044672513e7a
9,0x516008e8Db8CD5DC4D9784327C4b5F0890DF2287,825839,2019-01-03 22:12:31,1807755,0,15a01377585945e907654c4dba844944dad5206f998d09...,0x13495A46D4de87020753e3178462044672513e7a


Now let's look at *AuthCallBack* events posted by the Registry when it receives AuthCallback call from the Oracle

In [36]:
flt = escrowCode.events.AuthCallbackEvent.createFilter(fromBlock=1747955,toBlock=web3.eth.blockNumber)
ent = flt.get_all_entries()
ent = ent[:20]
len(ent)

20

In [37]:
auth_cb_events =  {n: {
    'id':"%x" % e.args['id'],
    'result': e.args['result'],
    'blk': e.blockNumber,
    'ts': datetime.utcfromtimestamp(web3.eth.getBlock(e.blockNumber)['timestamp']),
    'tx': e.transactionIndex} for n,e in enumerate(ent) }
auth_cb_events
auth_cb_events_df = pd.DataFrame.from_dict(auth_cb_events,orient='index')
auth_cb_events_df

,ts,tx,result,id,blk
0,2018-12-31 05:43:45,0,False,7ab926a9235011c8959e436024a12c3c0c1b118f0b77bc...,1747955
1,2018-12-31 06:05:51,0,True,b784c8325a15d7b7d62d4ded79b86b08fd0cbc8ed0099f...,1748206
2,2018-12-31 21:14:47,0,True,b784c8325a15d7b7d62d4ded79b86b08fd0cbc8ed0099f...,1758444
3,2019-01-02 01:47:15,0,True,691044a169e7fdda738ba4c7cc6d31b88d298546a326b7...,1777722
4,2019-01-02 06:55:13,0,True,691044a169e7fdda738ba4c7cc6d31b88d298546a326b7...,1781191
5,2019-01-02 21:29:23,0,True,691044a169e7fdda738ba4c7cc6d31b88d298546a326b7...,1791048
6,2019-01-02 23:58:48,0,True,691044a169e7fdda738ba4c7cc6d31b88d298546a326b7...,1792734
7,2019-01-04 00:25:00,0,False,b05d1d5c69459b0ef424a213f1c11d0afd53d070df8b69...,1809246
8,2019-01-04 00:40:53,0,False,b05d1d5c69459b0ef424a213f1c11d0afd53d070df8b69...,1809426
9,2019-01-04 00:49:41,0,False,b05d1d5c69459b0ef424a213f1c11d0afd53d070df8b69...,1809524


Here we see that Oracle usually response to login events in the next block of the chain.  For examlpe look for the block 1791047 in the AuthEvent table and then look for the next consecutive block 1791048 in the table above